<a href="https://colab.research.google.com/github/drfperez/algorithms/blob/main/Football.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# 🏟 Instal·lem llibreries
!pip install ultralytics opencv-python-headless deep_sort_realtime

# 🏟 Importem
import cv2
import pandas as pd
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from google.colab import files
import os

# 🏟 Pujar vídeo
uploaded = files.upload()
video_path = list(uploaded.keys())[0]
print(f'Fitxer pujat: {video_path}')

# 🏟 Carregar YOLOv8
model = YOLO('yolov8n.pt')

# 🏟 Configurar DeepSORT
tracker = DeepSort(max_age=30)

# 🏟 Obrir vídeo
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = '/content/output_tracked.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# 🏟 Inicialitzar DataFrame
data = []
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(frame, imgsz=640, conf=0.3, classes=[0])  # només persones
    detections = []

    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        confs = result.boxes.conf.cpu().numpy()
        for box, conf in zip(boxes, confs):
            x1, y1, x2, y2 = box
            detections.append(([x1, y1, x2 - x1, y2 - y1], conf, 'person'))

    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'ID {track_id}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        data.append({'frame': frame_count, 'track_id': track_id})

    out.write(frame)
    frame_count += 1
    if frame_count % 30 == 0:
        print(f'Frames processats: {frame_count}')

cap.release()
out.release()
cv2.destroyAllWindows()
print(f'Vídeo amb tracking guardat a: {output_path}')

# 🏟 Estadístiques bàsiques
df = pd.DataFrame(data)
counts = df.groupby('track_id')['frame'].nunique()
seconds_visible = counts / fps

print('\n⏱ Temps visible per jugador (segons):')
print(seconds_visible)

# 🏟 Permetre baixar el vídeo resultat
files.download(output_path)

In [ ]:

# 🏟 Instal·lar llibreries
!pip install ultralytics opencv-python-headless deep_sort_realtime seaborn

# 🏟 Importar llibreries
import cv2
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from google.colab import files
import os

# 🏟 Pujar vídeo (accepta .mp4 i .webm)
uploaded = files.upload()
video_path = list(uploaded.keys())[0]
print(f'Fitxer pujat: {video_path}')

# 🏟 Carregar YOLOv8
model = YOLO('yolov8n.pt')

# 🏟 Configurar DeepSORT
tracker = DeepSort(max_age=30)

# 🏟 Obrir vídeo
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = '/content/output_tracked.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# 🏟 Inicialitzar DataFrame per estadístiques i heatmap
data = []
positions = {}

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(frame, imgsz=640, conf=0.3, classes=[0])  # només persones
    detections = []

    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        confs = result.boxes.conf.cpu().numpy()
        for box, conf in zip(boxes, confs):
            x1, y1, x2, y2 = box
            detections.append(([x1, y1, x2 - x1, y2 - y1], conf, 'person'))

    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'ID {track_id}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        data.append({'frame': frame_count, 'track_id': track_id, 'cx': cx, 'cy': cy})

        # Guardar posicions per heatmap
        if track_id not in positions:
            positions[track_id] = []
        positions[track_id].append((cx, cy))

    out.write(frame)
    frame_count += 1
    if frame_count % 30 == 0:
        print(f'Frames processats: {frame_count}')

cap.release()
out.release()
cv2.destroyAllWindows()
print(f'\n✅ Vídeo amb tracking guardat a: {output_path}')

# 🏟 Estadístiques bàsiques
df = pd.DataFrame(data)
fps = fps if fps > 0 else 30  # fallback si FPS no llegit bé
counts = df.groupby('track_id')['frame'].nunique()
seconds_visible = counts / fps

print('\n⏱ Temps visible per jugador (segons):')
print(seconds_visible)

# 🏟 Generar heatmaps per jugador
for track_id, coords in positions.items():
    xs, ys = zip(*coords)
    heatmap, xedges, yedges = np.histogram2d(xs, ys, bins=(64, 64), range=[[0, width], [0, height]])
    heatmap = np.rot90(heatmap)
    heatmap = np.flipud(heatmap)
    plt.figure(figsize=(8, 6))
    sns.heatmap(heatmap, cmap='hot', cbar=True)
    plt.title(f'Heatmap jugador ID {track_id}')
    heatmap_path = f'/content/heatmap_{track_id}.png'
    plt.savefig(heatmap_path)
    plt.close()
    print(f'✅ Heatmap guardat: {heatmap_path}')

# 🏟 Permetre baixar fitxers resultants
files.download(output_path)
for track_id in positions.keys():
    files.download(f'/content/heatmap_{track_id}.png')

In [ ]:

# 🏗 Instal·lar llibreries
!pip install ultralytics opencv-python-headless deep_sort_realtime seaborn

# 🏗 Importar
import cv2
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from google.colab import files

# 🏗 Pujar vídeo
uploaded = files.upload()
video_path = list(uploaded.keys())[0]
print(f'Fitxer pujat: {video_path}')

# 🏗 Carregar YOLOv8
model = YOLO('yolov8n.pt')  # pots substituir per un model personalitzat si tens pilota/àrbitre entrenat

# 🏗 Configurar DeepSORT
tracker = DeepSort(max_age=30)

# 🏗 Obrir vídeo
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = '/content/output_tracked.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# 🏗 Inicialitzar dades
data = []
positions = {}
sector_counts = {}

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(frame, imgsz=640, conf=0.3)
    detections = []

    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        confs = result.boxes.conf.cpu().numpy()
        clss = result.boxes.cls.cpu().numpy()
        for box, conf, cls_id in zip(boxes, confs, clss):
            x1, y1, x2, y2 = box
            label = model.names[int(cls_id)]
            if label == 'person':  # només persones per DeepSORT
                detections.append(([x1, y1, x2 - x1, y2 - y1], conf, 'person'))
            # detecció especial (pilota, porteria, àrbitre)
            if label in ['sports ball', 'ball']:
                cv2.circle(frame, (int((x1 + x2)/2), int((y1 + y2)/2)), 10, (0, 0, 255), -1)
                cv2.putText(frame, 'Ball', (int(x1), int(y1 - 10)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            if label in ['goalpost', 'goal']:
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 255, 0), 2)
                cv2.putText(frame, 'Goal', (int(x1), int(y1 - 10)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
            if label in ['referee']:
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 255), 2)
                cv2.putText(frame, 'Referee', (int(x1), int(y1 - 10)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)

    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'ID {track_id}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        data.append({'frame': frame_count, 'track_id': track_id, 'cx': cx, 'cy': cy})

        # Guardar posicions per heatmap
        if track_id not in positions:
            positions[track_id] = []
        positions[track_id].append((cx, cy))

        # Comptar sectors tàctics
        sector_x = int(cx / (width / 3))
        sector_y = int(cy / (height / 3))
        sector = (sector_x, sector_y)
        if sector not in sector_counts:
            sector_counts[sector] = 0
        sector_counts[sector] += 1

    out.write(frame)
    frame_count += 1
    if frame_count % 30 == 0:
        print(f'Frames processats: {frame_count}')

cap.release()
out.release()
cv2.destroyAllWindows()
print(f'\n✅ Vídeo amb tracking guardat a: {output_path}')

# 🏗 Estadístiques bàsiques
df = pd.DataFrame(data)
fps = fps if fps > 0 else 30
counts = df.groupby('track_id')['frame'].nunique()
seconds_visible = counts / fps
print('\n⏱ Temps visible per jugador (segons):')
print(seconds_visible)

# 🏗 Heatmaps individuals i combinat
combined_heatmap = np.zeros((height, width))
for track_id, coords in positions.items():
    xs, ys = zip(*coords)
    heatmap, _, _ = np.histogram2d(xs, ys, bins=(width, height), range=[[0, width], [0, height]])
    combined_heatmap += heatmap
    plt.figure(figsize=(8, 6))
    sns.heatmap(np.flipud(heatmap.T), cmap='hot', cbar=True)
    plt.title(f'Heatmap jugador ID {track_id}')
    heatmap_path = f'/content/heatmap_{track_id}.png'
    plt.savefig(heatmap_path)
    plt.close()
    print(f'✅ Heatmap guardat: {heatmap_path}')

# 🏗 Heatmap general combinat
plt.figure(figsize=(10, 8))
sns.heatmap(np.flipud(combined_heatmap.T), cmap='hot', cbar=True)
plt.title('Heatmap general combinat')
combined_path = '/content/heatmap_combined.png'
plt.savefig(combined_path)
plt.close()
print(f'✅ Heatmap combinat guardat: {combined_path}')

# 🏗 Comptar presència per sectors
print('\n📊 Presència per sectors (3x3 grid):')
for sector, count in sorted(sector_counts.items()):
    print(f'Sector {sector}: {count} aparicions')

# 🏗 Permetre baixar fitxers
files.download(output_path)
for track_id in positions.keys():
    files.download(f'/content/heatmap_{track_id}.png')
files.download(combined_path)

In [ ]:

# 🏗 Instal·lar llibreries
!pip install ultralytics opencv-python-headless deep_sort_realtime seaborn

# 🏗 Importar
import cv2
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from google.colab import files

# 🏗 Pujar vídeo
uploaded = files.upload()
video_path = list(uploaded.keys())[0]
print(f'Fitxer pujat: {video_path}')

# 🏗 Carregar YOLOv8
model = YOLO('yolov8n.pt')

# 🏗 Configurar DeepSORT
tracker = DeepSort(max_age=30)

# 🏗 Obrir vídeo
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = '/content/output_tracked.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# 🏗 Inicialitzar dades
data = []
positions = {}
sector_counts = {}

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(frame, imgsz=640, conf=0.3)
    detections = []

    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        confs = result.boxes.conf.cpu().numpy()
        clss = result.boxes.cls.cpu().numpy()
        for box, conf, cls_id in zip(boxes, confs, clss):
            x1, y1, x2, y2 = box
            label = model.names[int(cls_id)]
            if label == 'person':
                detections.append(([x1, y1, x2 - x1, y2 - y1], conf, 'person'))
            if label in ['sports ball', 'ball']:
                cv2.circle(frame, (int((x1 + x2)/2), int((y1 + y2)/2)), 10, (0, 0, 255), -1)
                cv2.putText(frame, 'Ball', (int(x1), int(y1 - 10)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            if label in ['goalpost', 'goal']:
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 255, 0), 2)
                cv2.putText(frame, 'Goal', (int(x1), int(y1 - 10)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
            if label in ['referee']:
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 255), 2)
                cv2.putText(frame, 'Referee', (int(x1), int(y1 - 10)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)

    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'ID {track_id}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        data.append({'frame': frame_count, 'track_id': track_id, 'cx': cx, 'cy': cy})

        if track_id not in positions:
            positions[track_id] = []
        positions[track_id].append((cx, cy))

        sector_x = int(cx / (width / 3))
        sector_y = int(cy / (height / 3))
        sector = (sector_x, sector_y)
        if sector not in sector_counts:
            sector_counts[sector] = 0
        sector_counts[sector] += 1

    out.write(frame)
    frame_count += 1
    if frame_count % 30 == 0:
        print(f'Frames processats: {frame_count}')

cap.release()
out.release()
cv2.destroyAllWindows()
print(f'\n✅ Vídeo amb tracking guardat a: {output_path}')

# 🏗 Estadístiques
df = pd.DataFrame(data)
fps = fps if fps > 0 else 30
counts = df.groupby('track_id')['frame'].nunique()
seconds_visible = counts / fps
print('\n⏱ Temps visible per jugador (segons):')
print(seconds_visible)

# 🏗 Heatmaps
combined_heatmap = np.zeros((height, width))
for track_id, coords in positions.items():
    xs, ys = zip(*coords)
    heatmap, _, _ = np.histogram2d(xs, ys, bins=(width, height), range=[[0, width], [0, height]])
    combined_heatmap += heatmap
    plt.figure(figsize=(8, 6))
    sns.heatmap(np.flipud(heatmap.T), cmap='hot', cbar=True)
    plt.title(f'Heatmap jugador ID {track_id}')
    plt.savefig(f'/content/heatmap_{track_id}.png')
    plt.close()

plt.figure(figsize=(10, 8))
sns.heatmap(np.flipud(combined_heatmap.T), cmap='hot', cbar=True)
plt.title('Heatmap general combinat')
plt.savefig('/content/heatmap_combined.png')
plt.close()

# 🏗 Gràfic temps visible per jugador
plt.figure(figsize=(8, 6))
seconds_visible.sort_values(ascending=False).plot(kind='bar')
plt.ylabel('Segons visibles')
plt.title('Temps visible per jugador')
plt.tight_layout()
plt.savefig('/content/visible_time.png')
plt.close()
print(f'✅ Gràfic temps visible guardat: /content/visible_time.png')

# 🏗 Gràfic tàctic sectors (3x3)
sector_grid = np.zeros((3, 3))
for (sx, sy), count in sector_counts.items():
    sector_grid[sy, sx] = count

plt.figure(figsize=(6, 6))
sns.heatmap(np.flipud(sector_grid), annot=True, cmap='Blues', cbar=True, fmt='.0f')
plt.title('Presència per sectors (3x3)')
plt.savefig('/content/sector_presence.png')
plt.close()
print(f'✅ Gràfic sectors guardat: /content/sector_presence.png')

# 🏗 Gràfic de moviments superposat
plt.figure(figsize=(12, 8))
for track_id, coords in positions.items():
    xs, ys = zip(*coords)
    plt.plot(xs, ys, label=f'ID {track_id}', alpha=0.6)
plt.gca().invert_yaxis()
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Moviments combinats sobre el camp')
plt.legend()
plt.savefig('/content/movement_overlay.png')
plt.close()
print(f'✅ Gràfic moviments guardat: /content/movement_overlay.png')

# 🏗 Permetre baixar fitxers
files.download(output_path)
files.download('/content/heatmap_combined.png')
files.download('/content/visible_time.png')
files.download('/content/sector_presence.png')
files.download('/content/movement_overlay.png')
for track_id in positions.keys():
    files.download(f'/content/heatmap_{track_id}.png')

In [ ]:

# 🏗 Instal·lar llibreries
!pip install ultralytics opencv-python-headless deep_sort_realtime seaborn fpdf moviepy

# 🏗 Importar
import cv2
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from google.colab import files
from fpdf import FPDF
from moviepy.editor import VideoFileClip

# 🏗 Pujar vídeo
uploaded = files.upload()
video_path = list(uploaded.keys())[0]
print(f'Fitxer pujat: {video_path}')

# 🏗 Carregar YOLOv8
model = YOLO('yolov8n.pt')

# 🏗 Configurar DeepSORT
tracker = DeepSort(max_age=30)

# 🏗 Obrir vídeo
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = '/content/output_tracked.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# 🏗 Inicialitzar dades
data = []
positions = {}
sector_counts = {}

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(frame, imgsz=640, conf=0.3)
    detections = []

    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        confs = result.boxes.conf.cpu().numpy()
        clss = result.boxes.cls.cpu().numpy()
        for box, conf, cls_id in zip(boxes, confs, clss):
            x1, y1, x2, y2 = box
            label = model.names[int(cls_id)]
            if label == 'person':
                detections.append(([x1, y1, x2 - x1, y2 - y1], conf, 'person'))

    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'ID {track_id}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        data.append({'frame': frame_count, 'track_id': track_id, 'cx': cx, 'cy': cy})

        if track_id not in positions:
            positions[track_id] = []
        positions[track_id].append((cx, cy))

        sector_x = int(cx / (width / 3))
        sector_y = int(cy / (height / 3))
        sector = (sector_x, sector_y)
        if sector not in sector_counts:
            sector_counts[sector] = 0
        sector_counts[sector] += 1

    out.write(frame)
    frame_count += 1

cap.release()
out.release()
cv2.destroyAllWindows()
print(f'\n✅ Vídeo amb tracking guardat a: {output_path}')

# 🏗 Estadístiques
df = pd.DataFrame(data)
fps = fps if fps > 0 else 30
counts = df.groupby('track_id')['frame'].nunique()
seconds_visible = counts / fps

# 🏗 Gràfic comparatiu temps visible
plt.figure(figsize=(8, 6))
seconds_visible.sort_values(ascending=False).plot(kind='bar', color='skyblue')
plt.ylabel('Segons visibles')
plt.title('Comparativa temps visible per jugador')
plt.tight_layout()
plt.savefig('/content/visible_comparison.png')
plt.close()

# 🏗 Heatmap combinat
combined_heatmap = np.zeros((height, width))
for track_id, coords in positions.items():
    xs, ys = zip(*coords)
    heatmap, _, _ = np.histogram2d(xs, ys, bins=(width, height), range=[[0, width], [0, height]])
    combined_heatmap += heatmap

plt.figure(figsize=(10, 8))
sns.heatmap(np.flipud(combined_heatmap.T), cmap='hot', cbar=True)
plt.title('Heatmap general combinat')
plt.savefig('/content/heatmap_combined.png')
plt.close()

# 🏗 Gràfic sectors
sector_grid = np.zeros((3, 3))
for (sx, sy), count in sector_counts.items():
    sector_grid[sy, sx] = count

plt.figure(figsize=(6, 6))
sns.heatmap(np.flipud(sector_grid), annot=True, cmap='Blues', cbar=True, fmt='.0f')
plt.title('Presència per sectors (3x3)')
plt.savefig('/content/sector_presence.png')
plt.close()

# 🏗 Moviments overlay
plt.figure(figsize=(12, 8))
for track_id, coords in positions.items():
    xs, ys = zip(*coords)
    plt.plot(xs, ys, label=f'ID {track_id}', alpha=0.6)
plt.gca().invert_yaxis()
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Moviments combinats sobre el camp')
plt.legend()
plt.savefig('/content/movement_overlay.png')
plt.close()

# 🏗 Generar clips curts (top 10 sectors més actius)
top_sectors = sorted(sector_counts.items(), key=lambda x: x[1], reverse=True)[:3]
clip = VideoFileClip(output_path)
clip_duration = clip.duration

for i, (sector, _) in enumerate(top_sectors):
    start_time = max(0, (clip_duration / 10) * i)
    end_time = min(clip_duration, start_time + 5)
    subclip = clip.subclip(start_time, end_time)
    subclip_path = f'/content/top_clip_{i+1}.mp4'
    subclip.write_videofile(subclip_path, codec='libx264')
    print(f'✅ Clip generat: {subclip_path}')

# 🏗 Crear informe PDF
pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", 'B', 16)
pdf.cell(0, 10, 'Informe d\'Anàlisi de Partit', ln=True, align='C')

pdf.set_font("Arial", '', 12)
pdf.cell(0, 10, f'Total jugadors detectats: {len(positions)}', ln=True)
pdf.cell(0, 10, f'Total sectors actius: {len(sector_counts)}', ln=True)

pdf.image('/content/visible_comparison.png', w=180)
pdf.add_page()
pdf.image('/content/heatmap_combined.png', w=180)
pdf.add_page()
pdf.image('/content/sector_presence.png', w=180)
pdf.add_page()
pdf.image('/content/movement_overlay.png', w=180)

pdf_output = '/content/analysis_report.pdf'
pdf.output(pdf_output)
print(f'✅ Informe PDF generat: {pdf_output}')

# 🏗 Permetre baixar fitxers
files.download(output_path)
files.download(pdf_output)
files.download('/content/visible_comparison.png')
files.download('/content/heatmap_combined.png')
files.download('/content/sector_presence.png')
files.download('/content/movement_overlay.png')
for i in range(1, 4):
    files.download(f'/content/top_clip_{i}.mp4')

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 30.7 MB/s eta 0:00:00
  Created

  if event.key is 'enter':



Saving input.mp4 to input.mp4
Fitxer pujat: input.mp4


100%|██████████| 6.25M/6.25M [00:00<00:00, 18.4MB/s]


Streaming output truncated to the last 5000 lines.

0: 384x640 24 persons, 161.5ms
Speed: 8.9ms preprocess, 161.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 persons, 165.1ms
Speed: 2.6ms preprocess, 165.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 24 persons, 254.0ms
Speed: 2.4ms preprocess, 254.0ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 174.0ms
Speed: 2.5ms preprocess, 174.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 persons, 166.2ms
Speed: 2.5ms preprocess, 166.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 persons, 169.4ms
Speed: 2.5ms preprocess, 169.4ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 persons, 201.7ms
Speed: 2.3ms preprocess, 201.7ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 23 persons, 171.3ms
Speed: 

ValueError: operands could not be broadcast together with shapes (360,640) (640,360) (360,640) 